## This notebook deploys Sagemaker Endpoint / Transformer

### Step 1: Import Libraries
In this case we only need boto3 and Sagemaker libraries to prepare and deploy the models.

In [54]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
import tarfile
import boto3
print(sagemaker.__version__)

2.16.2.dev0


### Step 2: Prepare and upload trained LSTM model.
Tar up this model with inferencing script and pip requirements for sagemaker container. Upload trained model and inference script to s3 asset bucket to prepare for the deployment.

In [99]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
s3_bucket = 'reinvent2020-anomaly-prediction-{}-us-west-2-dev'.format(account_id)
model_pth_file_path = './model.pth'
model_packaged_name = 'model.tar.gz'
tar = tarfile.open(model_packaged_name, "w:gz")
tar.add(model_pth_file_path)
tar.add('code/requirements.txt')
tar.close()

s3_client = boto3.client('s3')
try:
    response = s3_client.upload_file(model_packaged_name, s3_bucket, 'trained_models/{}'.format(model_packaged_name))
except ClientError as e:
    logging.error(e)


### Step 3: Deploy model with Sagemaker container
IN this step. we are going to run deployment script to deploy the model we have prepared in the previous step and launch a Sagemaker container to host this model and serve live inferencing on Sagemaker endpoint.

In [98]:
role = get_execution_role() 
pytorch_model = PyTorchModel(model_data='s3://{}/trained_models/{}'.format(s3_bucket, model_packaged_name), role=role, entry_point='code/inference.py', framework_version='1.6.0', py_version='py3')
predictor = pytorch_model.deploy(instance_type='ml.t2.medium', initial_instance_count=1)


-------------------!